In [1]:
import streamlit as st
import pandas as pd
import fbprophet
from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation
from fbprophet.plot import plot_cross_validation_metric

In [3]:
df = pd.read_csv("Data/temp_clean.csv")
df.rename(columns={'date': 'ds', 'Avg_anomalies': 'y'}, inplace=True)


max_date = df['ds'].max() #compute latest date in the data 



obj = Prophet()
obj.fit(df)

future = obj.make_future_dataframe(periods=30)

fcst = obj.predict(future)  #make prediction for the extended data
forecast = fcst[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]




2022-07-08 22:17:57.250 INFO    fbprophet: Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-07-08 22:17:57.251 INFO    fbprophet: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -54.0013
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2784.17    0.00211607       110.744      0.6503      0.6503      111   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     169        2794.4   0.000669046       129.122   2.742e-06       0.001      231  LS failed, Hessian reset 
     199       2797.26   2.05312e-05       57.2649       0.142       0.142      266   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     277       2798.82    0.00056925       132.605   4.177e-06       0.001      396  LS failed, Hessian reset 
     294       2799.28   1.82064e-09       98.2704   1.465e-07     0.09791      431   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


In [4]:
forecast.head()

,ds,yhat,yhat_lower,yhat_upper
0,1880-01-31,-0.226063,-0.405996,-0.035196
1,1880-02-29,-0.174176,-0.364960,0.021414
2,1880-03-31,-0.167839,-0.349352,0.026259
3,1880-04-30,-0.207849,-0.392284,-0.022960
4,1880-05-31,-0.208721,-0.386486,-0.025357


In [5]:
df.head()

,ds,y
0,1880-01-31,-0.29
1,1880-02-29,-0.18
2,1880-03-31,-0.11
3,1880-04-30,-0.20
4,1880-05-31,-0.12


In [7]:
df.shape

(1668, 2)

In [6]:
df.head(df.shape[0])

,ds,y
0,1880-01-31,-0.29
1,1880-02-29,-0.18
2,1880-03-31,-0.11
3,1880-04-30,-0.20
4,1880-05-31,-0.12
...,...,...
1663,2018-08-31,0.73
1664,2018-09-30,0.76
1665,2018-10-31,0.99
1666,2018-11-30,0.78


In [9]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(df["y"], forecast["yhat"].head(df.shape[0]), squared=False)

In [10]:
rms

0.14626400084718297

In [11]:
import matplotlib.pyplot as plt

In [12]:
import plotly.express as px 

# Plot 
fig = px.line(df, x='ds', y='y')

# Only thing I figured is - I could do this 
fig.add_scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines')

# Show plot 
fig.show()

In [40]:
from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation
from fbprophet.plot import plot_cross_validation_metric
from fbprophet.diagnostics import cross_validation


In [41]:
df = pd.read_csv("Data/temp_clean.csv")
df.rename(columns={'date': 'ds', 'Avg_anomalies': 'y'}, inplace=True)


max_date = df['ds'].max() #compute latest date in the data 

#periods_input = st.number_input('How many days forecast do you want?', min_value = 15, max_value = 365)
periods_input =30



obj = Prophet()
obj.fit(df)
future = obj.make_future_dataframe(periods=int(periods_input))

fcst = obj.predict(future)  #make prediction for the extended data
forecast = fcst[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


rms = round(mean_squared_error(df["y"], forecast["yhat"].head(df.shape[0]), squared=False), 2)        



2022-07-09 00:18:50.637 Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-07-09 00:18:50.650 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -54.0013
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2784.17    0.00211607       110.744      0.6503      0.6503      111   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     169        2794.4   0.000669046       129.122   2.742e-06       0.001      231  LS failed, Hessian reset 
     199       2797.26   2.05312e-05       57.2649       0.142       0.142      266   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     277       2798.82    0.00056925       132.605   4.177e-06       0.001      396  LS failed, Hessian reset 
     294       2799.28   1.82064e-09       98.2704   1.465e-07     0.09791      431   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


Here I expect to evaluate on 1-day forecast from the 2189th row of the data to the end of it. So it should:

1 - Create a model with 100 datapoints as training data and evaluate on the 200th datapoint  
2 - Create a model with 200 datapoints as training data and evaluate on the 300th datapoint  
3 - Create a model with 300 datapoints as training data and evaluate on the 400th datapoint  
.  
.  
.  
N - Create a model with N-1 datapoints as training data and evaluate on the Nth (last) datapoint.

In [42]:
df.shape

(1668, 2)

In [81]:
df.head()

,ds,y
0,1880-01-31,-0.29
1,1880-02-29,-0.18
2,1880-03-31,-0.11
3,1880-04-30,-0.20
4,1880-05-31,-0.12


In [43]:
cutoffs = pd.to_datetime(['1918-09-3', '1958-09-3', '2018-09-3'])

df_cv2 = cross_validation(obj, cutoffs=cutoffs, horizon='365 days')


  0%|          | 0/3 [00:00<?, ?it/s]

Initial log joint probability = -11.775
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       534.837   4.31254e-06       89.5858      0.4181           1      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     133       534.837   2.14465e-08       83.5599      0.3297      0.3297      173   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


 33%|███▎      | 1/3 [00:02<00:04,  2.11s/it]

Initial log joint probability = -23.9349
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      67        1069.5     0.0186663       94.2898   0.0001631       0.001      109  LS failed, Hessian reset 
      99       1070.58    0.00195244       80.0492      0.1136      0.7338      153   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     100       1070.59    0.00137581       86.8017   1.719e-05       0.001      189  LS failed, Hessian reset 
     157       1070.88   0.000209412       87.6259   2.561e-06       0.001      293  LS failed, Hessian reset 
     199       1070.92    0.00160212       95.0059           1           1      340   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     214       1071.04   0.000244732        85.992    3.29e-06       0.001      395  LS failed, Hessian reset 
     299       1071.21   0.000458672       70.7158      0.7321   

 67%|██████▋   | 2/3 [00:04<00:02,  2.51s/it]

Initial log joint probability = -36.4301
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2775.41    0.00424772       185.059      0.7873      0.7873      119   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     147       2787.96   0.000841108       168.636   6.573e-06       0.001      214  LS failed, Hessian reset 
     199       2789.42   5.83896e-06       63.7373      0.7675      0.7675      287   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     248       2789.62    0.00150409       207.051   1.278e-05       0.001      393  LS failed, Hessian reset 
     299       2789.88   1.21726e-06        63.576      0.5859      0.5859      462   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       2791.67   2.31242e-05       70.6859           1           1      587   
    Iter      log pro

100%|██████████| 3/3 [00:07<00:00,  2.51s/it]


In [67]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv2)
df_p.head()

2022-07-09 00:36:58.963 Skipping MAPE because y close to 0


,horizon,mse,rmse,mae,mdape,coverage
0,27 days,0.011833,0.108778,0.091371,1.151745,1.000000
1,58 days,0.031494,0.177465,0.140365,0.167285,0.666667
2,88 days,0.019278,0.138844,0.099990,0.368040,0.666667
3,119 days,0.006530,0.080811,0.076072,0.370586,1.000000
4,150 days,0.029176,0.170809,0.156714,1.116461,0.500000


In [68]:
df_p = round(df_p, 3)
df_p["horizon"] = df_p["horizon"].astype(str)

In [69]:
df_p.head(2)

,horizon,mse,rmse,mae,mdape,coverage
0,27 days,0.012,0.109,0.091,1.152,1.000
1,58 days,0.031,0.177,0.140,0.167,0.667


In [70]:
df_p["horizon"].unique()

array(['27 days', '58 days', '88 days', '119 days', '150 days',
       '178 days', '209 days', '239 days', '270 days', '300 days',
       '331 days', '362 days'], dtype=object)

In [80]:
fig = px.bar(df_p, x = "horizon", y = "mse",color = "horizon" , template='plotly_white', text='mse', color_discrete_sequence=px.colors.qualitative.Prism,
)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False, marker_line_width=1.5, opacity=1)


In [76]:
fig = px.colors.qualitative.swatches()
fig.show()